In [ ]:
import torch 
from torch import nn, optim 
import torch.nn.functional as F 

import numpy as np 
import pandas as pd 
import math
from tqdm import tqdm 

from torch.utils.data import Dataset, DataLoader 

from sklearn.preprocessing import StandardScaler

import os 


from model import * 
# from modeling import * 
from utils import * 

In [ ]:
class args:
    win_size = 100 
    input_c = 25
    output_c = 25
    num_layers = 3
    num_epochs = 200
    batch_size = 256 
    num_workers = 0
    lambda_ = 3
    lr = 1e-4
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
# train_loader = get_loader(args, mode='train')
# test_loader = get_loader(args, mode='test')

In [ ]:
class PSMSegLoader(object):
    def __init__(self, data_path, win_size, step, mode="train"):
        self.mode = mode
        self.step = step
        self.win_size = win_size
        self.scaler = StandardScaler()
        data = pd.read_csv(data_path + '/data/train.csv')
        data = data.values[:, 1:]

        data = np.nan_to_num(data)

        self.scaler.fit(data)
        data = self.scaler.transform(data)
        test_data = pd.read_csv(data_path + '/data/test.csv')

        test_data = test_data.values[:, 1:]
        test_data = np.nan_to_num(test_data)

        self.test = self.scaler.transform(test_data)

        self.train = data
        self.val = self.test

        self.test_labels = pd.read_csv(data_path + '/data/test_label.csv').values[:, 1:]

        print("test:", self.test.shape)
        print("train:", self.train.shape)

    def __len__(self):
        """
        Number of images in the object dataset.
        """
        if self.mode == "train":
            return (self.train.shape[0] - self.win_size) // self.step + 1
        elif (self.mode == 'val'):
            return (self.val.shape[0] - self.win_size) // self.step + 1
        elif (self.mode == 'test'):
            return (self.test.shape[0] - self.win_size) // self.step + 1
        else:
            return (self.test.shape[0] - self.win_size) // self.win_size + 1

    def __getitem__(self, index):
        index = index * self.step
        if self.mode == "train":
            return np.float32(self.train[index:index + self.win_size]), np.float32(self.test_labels[0:self.win_size])
        elif (self.mode == 'val'):
            return np.float32(self.val[index:index + self.win_size]), np.float32(self.test_labels[0:self.win_size])
        elif (self.mode == 'test'):
            return np.float32(self.test[index:index + self.win_size]), np.float32(
                self.test_labels[index:index + self.win_size])
        else:
            return np.float32(self.test[
                              index // self.step * self.win_size:index // self.step * self.win_size + self.win_size]), np.float32(
                self.test_labels[index // self.step * self.win_size:index // self.step * self.win_size + self.win_size])

In [ ]:
train_set = PSMSegLoader('.', args.win_size, 1, mode='train')
test_set = PSMSegLoader('.', args.win_size, 1, mode='test')

train_loader = DataLoader(train_set, batch_size=args.batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=args.batch_size, shuffle=False)

In [ ]:
model = AnomalyTransformer(
    window_size = args.win_size, 
    enc_in = args.input_c, 
    c_out = args.output_c, 
    e_layers = args.num_layers
).to(args.device)

optimizer = optim.Adam(model.parameters(), lr=args.lr)
criterion = nn.MSELoss().to(args.device)

In [ ]:
def evaluate( model, valid_loader, criterion):
    
    model.eval()
    
    min_losses, max_losses = [], []
    
    for batch in valid_loader:
        input_x, label = tuple(b.to(args.device) for b in batch)
        
        series_loss, prior_loss = 0.0, 0.0
        
        x_hat, series, prior, _ = model(input_x)
        
        for i in range(len(prior)):
            series_loss += (torch.mean(F.kl_div(series[i], (
                prior[i] / torch.unsqueeze(torch.sum(prior[i], dim=-1), dim=-1).repeat(1, 1, 1, args.win_size)).detach())) + torch.mean(
                
                F.kl_div(
                    (prior[i] / torch.unsqueeze(torch.sum(prior[i], dim=-1), dim=-1).repeat(1, 1, 1, args.win_size)).detach(), series[i])
            ))
                
            prior_loss += (torch.mean(
                F.kl_div((prior[i] / torch.unsqueeze(torch.sum(prior[i], dim=-1), dim=-1).repeat(1, 1, 1, args.win_size)), 
                             series[i].detach())) + torch.mean(
                F.kl_div(series[i].detach(), 
                             (prior[i] / torch.unsqueeze(torch.sum(prior[i], dim=-1), dim=-1).repeat(1, 1, 1, args.win_size)))))
            
        series_loss = series_loss / len(prior)
        prior_loss = prior_loss / len(prior)
        reconstruction_loss = criterion(x_hat, input_x)
        
        
        max_loss = reconstruction_loss - args.lambda_ * prior_loss
        min_loss = reconstruction_loss + args.lambda_ * series_loss 
    
    return np.average(min_losses), np.average(max_losses)

In [ ]:
iterator = tqdm(range(args.num_epochs), desc='training...')
for epoch in iterator:
    min_best_loss = float('inf')
    max_best_loss = float('inf')
    
    model.train()
    for idx, batch in enumerate(train_loader):
        train_losses = 0.0
        input_x, label = tuple(b.to(args.device) for b in batch)
        
        optimizer.zero_grad()
        input_x = input_x.float().to(args.device)
        
        x_hat, series, prior, _ = model(input_x) # (x_hat, series, prior, sigma)
        
        series_loss, prior_loss = 0.0, 0.0 
        for i in range(len(prior)):
            series_loss += (torch.mean(F.kl_div(series[i], (
                prior[i] / torch.unsqueeze(torch.sum(prior[i], dim=-1), dim=-1).repeat(1, 1, 1, args.win_size)).detach())) + torch.mean(
                
                F.kl_div(
                    (prior[i] / torch.unsqueeze(torch.sum(prior[i], dim=-1), dim=-1).repeat(1, 1, 1, args.win_size)).detach(), series[i])
            ))
                
            prior_loss += (torch.mean(
                F.kl_div((prior[i] / torch.unsqueeze(torch.sum(prior[i], dim=-1), dim=-1).repeat(1, 1, 1, args.win_size)), 
                             series[i].detach())) + torch.mean(
                F.kl_div(series[i].detach(), 
                             (prior[i] / torch.unsqueeze(torch.sum(prior[i], dim=-1), dim=-1).repeat(1, 1, 1, args.win_size)))))
                
        series_loss = series_loss / len(prior)
        prior_loss = prior_loss / len(prior)
        reconstruction_loss = criterion(x_hat, input_x)

        max_loss = reconstruction_loss - args.lambda_ * prior_loss
        min_loss = reconstruction_loss + args.lambda_ * series_loss 
        
        iterator.set_postfix({
            'min_loss':min_loss.item(), 
            'max_loss':max_loss.item(), 
            'rec_loss':reconstruction_loss.item()})
        
        if (idx + 1 % 50) == 0 :
            print(f'Epoch: [{epoch+1}/{args.num_epochs}]\t iterator: [{idx+1}/{len(train_loader)}]')
            print(f'max loss: {max_loss:.4f},\tmin_loss: {min_loss:.4f},\treconstruction loss: {reconstruction_loss:.4f}')
        
        # Minimax strategy
        max_loss.backward(retain_graph=True)
        min_loss.backward()
        optimizer.step()
    
    
    valid_min_loss, valid_max_loss = evaluate(model, test_loader, criterion)